In [1]:
!pip install gymnasium numpy # Install necessary packages

In [2]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces
import time
from IPython.display import clear_output
import sys

In [3]:
class GridEnv(gym.Env):
    """
    A simple GridWorld environment compatible with Gymnasium and Google Colab (text-based rendering).
    Agent starts at (0, 0) and goal is at (size-1, size-1).
    """
    metadata = {"render_modes": ["human", "ansi"], "render_fps": 4} # Use 'ansi' for text rendering in Colab

    def __init__(self, render_mode="ansi", grid_size=5):
        super().__init__()
        self.grid_size = grid_size
        self.render_mode = render_mode
        self.agent_location = np.array([0, 0])
        self.goal_location = np.array([grid_size - 1, grid_size - 1])
        self.max_episode_steps = grid_size * 4 # Limit steps to prevent infinite wandering
        self.current_step = 0

        # Define action and observation space
        # Actions: 0: right, 1: up, 2: left, 3: down
        self.action_space = spaces.Discrete(4)
        # Observations: Agent location (x, y)
        self.observation_space = spaces.Box(low=0, high=self.grid_size - 1,
                                            shape=(2,), dtype=int)

    def _get_obs(self):
        return self.agent_location

    def _get_info(self):
        return {"distance_to_goal": np.linalg.norm(self.agent_location - self.goal_location, ord=1)}

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.agent_location = np.array([0, 0]) # Agent starts at top-left
        self.current_step = 0
        observation = self._get_obs()
        info = self._get_info()
        return observation, info

    def step(self, action):
        self.current_step += 1
        # Map actions to movements: 0: right, 1: up, 2: left, 3: down
        if action == 0:
            self.agent_location[0] += 1
        elif action == 1:
            self.agent_location[1] -= 1
        elif action == 2:
            self.agent_location[0] -= 1
        elif action == 3:
            self.agent_location[1] += 1

        # Clip movement to stay within the grid boundaries
        self.agent_location = np.clip(self.agent_location, 0, self.grid_size - 1)

        # Check if goal reached
        terminated = np.array_equal(self.agent_location, self.goal_location)
        reward = 1.0 if terminated else 0.0

        # Check for truncation (episode length limit)
        truncated = self.current_step >= self.max_episode_steps

        observation = self._get_obs()
        info = self._get_info()

        return observation, reward, terminated, truncated, info

    def render(self):
        if self.render_mode == "ansi":
            clear_output(wait=True)
            grid = [["⬜" for _ in range(self.grid_size)] for _ in range(self.grid_size)]

            # Place goal
            gx, gy = self.goal_location
            grid[gy][gx] = "⭐"

            # Place agent
            ax, ay = self.agent_location
            grid[ay][ax] = "🤖"

            # Print grid
            for row in grid:
                print("".join(row))
            time.sleep(0.5) # Pause to make visualization discernible
        else:
            # Placeholder for potential Pygame or other rendering in a local env
            print(f"Agent at {self.agent_location}, Goal at {self.goal_location}")


    def close(self):
        pass # No resources to close with text rendering

# --- Example Usage in Google Colab ---
env = GridEnv(grid_size=5, render_mode="ansi")
observation, info = env.reset()
terminated, truncated = False, False

print("Initial Observation:", observation)

while not terminated and not truncated:
    action = env.action_space.sample() # Take a random action
    observation, reward, terminated, truncated, info = env.step(action)
    env.render()
    print(f"Action taken: {action}, Reward: {reward}, Terminated: {terminated}, Truncated: {truncated}")

env.close()


⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜
⬜⬜⬜⬜🤖
⬜⬜⬜⬜⭐
Action taken: 3, Reward: 0.0, Terminated: False, Truncated: True
